<a href="https://colab.research.google.com/github/elangbijak4/Blockchain_Research/blob/main/Blockchain_PoA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import time
import datetime
import json

In [ ]:
class Block:
    def __init__(self, index, previous_hash, timestamp, data, validator):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.validator = validator
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = f"{self.index}{self.previous_hash}{self.timestamp}{self.data}{self.validator}"
        return hashlib.sha256(block_string.encode()).hexdigest()

    def to_dict(self):
        return {
            'index': self.index,
            'previous_hash': self.previous_hash,
            'timestamp': self.timestamp,
            'data': self.data,
            'validator': self.validator,
            'hash': self.hash
        }

In [ ]:
class Blockchain:
    def __init__(self, validators, filename='blockchain.json'):
        self.validators = validators
        self.filename = filename
        try:
            with open(self.filename, 'r') as f:
                self.chain = self.load_from_json(f.read())
        except FileNotFoundError:
            self.chain = [self.create_genesis_block()]
            self.save_to_json()

    def create_genesis_block(self):
        return Block(0, "0", self.get_utc_time(), "Genesis Block", self.validators[0])

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, data):
        previous_block = self.get_latest_block()
        validator = self.select_validator()
        new_block = Block(len(self.chain), previous_block.hash, self.get_utc_time(), data, validator)
        self.chain.append(new_block)
        self.save_to_json()

    def select_validator(self):
        # Dalam implementasi nyata, pemilihan validator harus berdasarkan aturan tertentu
        return self.validators[len(self.chain) % len(self.validators)]

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            if current_block.hash != current_block.calculate_hash():
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True

    def get_utc_time(self):
        return datetime.datetime.utcnow().timestamp()

    def display_chain(self):
        with open(self.filename, 'r') as f:
            blockchain_data = json.load(f)
            for block_data in blockchain_data:
                # Remove 'hash' from block_data as it's calculated internally
                block_data.pop('hash', None)
                block = Block(**block_data)  # Membuat objek Block dari data JSON
                print(f"Block {block.index}:")
                print(f"  Timestamp: {datetime.datetime.utcfromtimestamp(block.timestamp)}")
                print(f"  Data: {block.data}")
                print(f"  Validator: {block.validator}")
                print(f"  Hash: {block.hash}")
                print(f"  Previous Hash: {block.previous_hash}\n")

    def save_to_json(self):
        with open(self.filename, 'w') as f:
            json.dump([block.to_dict() for block in self.chain], f, indent=4)

    def load_from_json(self, json_string):
        data = json.loads(json_string)
        blocks = []
        for block_data in data:
            # Remove 'hash' from block_data as it's calculated internally
            block_data.pop('hash', None)
            block = Block(**block_data)  # Membuat objek Block dari data JSON
            blocks.append(block)
        return blocks

In [ ]:
# Implementasi Wewenang (Authority)
validators = ["Adil", "Rimba", "Nita"]
blockchain = Blockchain(validators)

In [ ]:
blockchain.add_block("Data untuk blok 1")
blockchain.add_block("Data untuk blok 2")
blockchain.add_block("Data untuk blok 3")

In [ ]:
blockchain.display_chain()

Block 0:
  Timestamp: 2024-06-24 11:33:50.590884
  Data: Genesis Block
  Validator: Adil
  Hash: b68079b2f6f7866fa286155b39e55837cc10199de00d673047026dbee5788d70
  Previous Hash: 0

Block 1:
  Timestamp: 2024-06-24 11:34:10.358973
  Data: Data untuk blok 1
  Validator: Rimba
  Hash: 2132147a361f62423c0e8533fd2fcef832e1184e99cef92f2cefd0a9b7f33d47
  Previous Hash: b68079b2f6f7866fa286155b39e55837cc10199de00d673047026dbee5788d70

Block 2:
  Timestamp: 2024-06-24 11:34:10.359674
  Data: Data untuk blok 2
  Validator: Nita
  Hash: 5b89ae13d380e97b735ff8a3abe685dce52f52554029c2ef898271dc04137b1c
  Previous Hash: 2132147a361f62423c0e8533fd2fcef832e1184e99cef92f2cefd0a9b7f33d47

Block 3:
  Timestamp: 2024-06-24 11:34:10.360187
  Data: Data untuk blok 3
  Validator: Adil
  Hash: 35489e2319e47c03984b4381206bde85d5068701655dea99e6cd8a46926def98
  Previous Hash: 5b89ae13d380e97b735ff8a3abe685dce52f52554029c2ef898271dc04137b1c

